# Virtual Try-On: Mask Shape

The `maskShape` parameter controls the shape of any auto-generated mask (created with either the `"GARMENT"` or `"PROMPT"` `maskType`). It offers `"CONTOUR"` and `"BOUNDING_BOX"` options.

When using the `"GARMENT"` mask type, Nova Canvas creates a mask that adheres closely to the body contour of the person in the source image by default. There are some cases where this default behavior is not desirable, producing an inaccurate fit, such as:

- Using a loose fitting dress as the reference image
- Adding pants to an image of someone wearing leggings or shorts
- Swapping shoes/boots of a significantly different style (e.g. high heels to flats)

For these cases, you can set the `maskShape` paremeter to `"BOUNDING_BOX"` for better results.

**Input**:

<div style="display: flex; gap: 2em; max-height: 500px; margin-right: 2em; margin-bottom: 1em">
  <div style="display: flex; flex-direction: column; gap: 1em; align-items: center; max-width: 300px">
    <img src="../images/woman-partial_body-2.png">
    <strong><tt>sourceImage</tt></strong>
  </div>
  <div style="display: flex; flex-direction: column; gap: 1em; align-items: center; max-width: 300px">
    <img src="../images/dress-blue.png">
    <strong><tt>referenceImage</tt></strong>
  </div>
</div>

**Output**:

<div style="display: flex; gap: 2em; max-height: 500px; margin-right: 2em; margin-bottom: 1em">
  <div style="display: flex; flex-direction: column; gap: 1em; align-items: center; max-width: 300px">
    <img src="../images/mask_shape_contour_output.png">
    <code>maskShape: "CONTOUR"</code> (default)
  </div>
  <div style="display: flex; flex-direction: column; gap: 1em; align-items: center; max-width: 300px">
    <img src="../images/mask_shape_bbox_output.png">
    <code>maskShape: "BOUNDING_BOX"</code>
  </div>
</div>

_✏️ **NOTE:** When using the `"BOUNDING_BOX"` mask shape, the pose of the person in the source image may change. This is because the bounding box mask essentially "hides" the original pose details from Nova Canvas. In cases where you want to preserve the original pose <u>and</u> use a bounding box mask, you can achieve this using the `maskExclusions.preserveBodyPose` parameter discussed in a later notebook._

_⚠️ **WARNING:** In this example, you may have noticed the the `"BOUNDING_BOX"` exhibits a slight rectangular discoloration of the background, making the rectangular mask area perceptible. This is an artifact that can occur when using the bounding box shape and is more likely to occur when the background is a solid color. To avoid this, you can use a `mergeStyle` parameter of `"SEAMLESS"` which is demonstrated later in this notebook._


In [ ]:
import logging
import os
from datetime import datetime
from random import randint
from amazon_nova_canvas_utils import (
    generate_images,
    load_image_as_base64,
    base64_to_pil_image,
)

logging.basicConfig(format="[%(levelname)s] %(message)s", level=logging.INFO)

# Edit these values to experiment with your own images.
source_image_path = "../images/vto-images/vto_mask_shape_source.jpg"
reference_image_path = "../images/vto-images/vto_mask_shape_reference.jpg"

inference_params = {
    "taskType": "VIRTUAL_TRY_ON",
    "virtualTryOnParams": {
        "sourceImage": load_image_as_base64(source_image_path),
        "referenceImage": load_image_as_base64(reference_image_path),
        "maskType": "GARMENT",
        "garmentBasedMask": {
            "garmentClass": "FULL_BODY",
            # Added maskShape set to "BOUNDING_BOX"
            "maskShape": "BOUNDING_BOX",
        },
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 6.5,
        "seed": randint(0, 2147483646),
    },
}

output_folder = os.path.join("output", datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

try:
    response_body = generate_images(
        inference_params=inference_params,
        save_folder_path=output_folder,
        model_id="amazon.nova-canvas-v1:0",
        region_name="us-east-1",
    )

    # An error message may be returned, even if some images were generated.
    if "error" in response_body:
        logging.error(response_body["error"])

    if "images" in response_body:
        # Display all images.
        for image_base64 in response_body["images"]:
            image = base64_to_pil_image(image_base64)
            display(image)

except Exception as e:
    logging.error(e)

print(f"Done! Artifacts saved to {os.path.abspath(output_folder)}")